In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from matplotlib import pyplot as plt
import pandas as pd
import datetime

In [2]:
def featurize(df, label):
  # Set label and features
  df_copy = df.copy()
  y = df_copy[label]
  X = df_copy.assign(const=1)  # drop all categorical features and allow y-intercept to vary
  X = X.drop(columns=[label])
  return y, X

def non_numeric_dtypes(df):
  non_numeric = list(set(df.columns) - set(df.select_dtypes('number').columns))
  print(non_numeric)
  return non_numeric

def group_by_counts(df, labels):
  df_copy = df.copy()
  df_copy = df_copy.groupby(labels).size.reset_index().rename(columns={0:'count'})
  return df_copy

In [9]:
df = pd.read_csv("noReplies-clean.csv")
df_pre = pd.read_csv("noRepies-topics.csv")
df["handle"] = df_pre["handle"]
df.head()

,Unnamed: 0,Unnamed: 0.1,created_at,num_referenced_tweets,topic_1,topic_2,topic_3,topic_4,num_hashtags_ln,num_mentions_ln,...,source_Sprout Social,source_Twitter Web App,source_Twitter Web Client,source_Twitter for iPhone,source_other,is_retweet_True,is_reply_True,contains_quote_True,has_image_True,handle
0,0,0,692,0,0.052988,0.053030,0.594013,0.299969,0.0,0.0,...,0,0,0,0,1,0,0,0,0,Applebees
1,1,1,1009,0,0.624984,0.125005,0.125006,0.125005,0.0,0.0,...,0,0,0,0,1,0,0,0,0,Applebees
2,2,2,1009,0,0.178700,0.246201,0.035738,0.539361,0.0,0.0,...,0,0,0,0,1,0,0,0,0,Applebees
3,3,3,1009,0,0.062553,0.062553,0.562856,0.312038,0.0,0.0,...,0,0,0,0,1,0,0,0,0,Applebees
4,4,4,1009,0,0.296835,0.043752,0.134403,0.525009,0.0,0.0,...,0,0,0,0,1,0,0,0,0,Applebees


In [10]:
y, X = featurize(df, "handle")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
clf = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = clf.predict(X_test)

results = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred})

In [21]:
results["Correct"] = (results["Actual"] == results["Predicted"])
results["Correct"].value_counts()

True     29954
False       44
Name: Correct, dtype: int64